## Module import

In [1]:
import sqlite3
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import plotly
import plotly.graph_objects as go
import ipywidgets as widgets

## Module version

In [2]:
print("plotly version:", plotly.__version__)
print("matplotlib version:", matplotlib.__version__)
print("sqlite3 version:", sqlite3.version)
print("pandas version:", pd.__version__)
print("numpy version:", np.__version__)
print("ipywidgets version:", widgets.__version__)

plotly version: 4.14.3
matplotlib version: 3.3.2
sqlite3 version: 2.6.0
pandas version: 1.2.1
numpy version: 1.19.2
ipywidgets version: 7.5.1


# Chromosome visualisation

## Functions

In [3]:
#Chromosome data recuperation

def stringToList(string):
    return list(string.split(", "))

def getChromInfo(database, chrom_number, info):
    #SQL request
    db_connexion = sqlite3.connect(database)
    
    cursor = db_connexion.cursor()
    
    chrom_sense = cursor.execute("""
    SELECT """+ info +"""
    FROM SGD_features 
    WHERE Chromosome == """+ chrom_number +""" AND Strand == 'W'
    ORDER BY Start_coordinate
    """)
    
    cursor = db_connexion.cursor()

    chrom_antisense = cursor.execute("""
    SELECT """+ info +"""
    FROM SGD_features 
    WHERE Chromosome == """+ chrom_number +""" AND Strand == 'C'
    ORDER BY Stop_coordinate
    """)
    
    List_info = stringToList(info)
    
    #Pandas dataframe formatting
    chrom_sense = chrom_sense.fetchall()
    chrom_sense = pd.DataFrame(chrom_sense, columns=List_info)
    
    chrom_antisense = chrom_antisense.fetchall()
    chrom_antisense = pd.DataFrame(chrom_antisense, columns=List_info)
    
    return chrom_sense, chrom_antisense

In [4]:
#Chromosomes drawing

def AddChromosome(fig, x1, x2, y1, y2, color, number):
    
    chromosome = go.Scatter(
        x = [x1,x2,x2,x1,x1], 
        y = [y1,y1,y2,y2,y1],
        hoverinfo = 'none',
        marker_symbol = "diamond-tall",
        marker_size = 0.1,
        showlegend = False,
        line_color = color,
        line_width = 0.1,
        fillcolor = color,
        fill = "toself")
    fig.add_trace(chromosome)
    
    chromosome_legend = go.Scatter(
        x = [x1+17000, x2+10000, x2+13000],
        y = [y1+10, (y2/4), 3*(y2/4)],
        text = ["Chromosome " + number, "Sense", "Antisense"],
        mode = "text",
        showlegend = False,
        hoverinfo = 'none')
    fig.add_trace(chromosome_legend)

In [5]:
#Locus drawing

def AddLocus(chrom, fig, y1, y2, parameter, value, color):
    for p,v,c in zip(parameter,value,color) :
        if chrom[p] == v :
            locus = go.Scatter(
                x = [chrom['Start_coordinate'],
                     chrom['Start_coordinate'],
                     chrom['Stop_coordinate'],
                     chrom['Stop_coordinate'],
                     chrom['Start_coordinate']], 
                y = [y1,y2,y2,y1,y1],
                name = chrom['Primary_SGDID'],
                hoveron = 'fills',
                hovertemplate = "<b>ID:%{name}</b>",
                marker_symbol = "diamond-tall",
                marker_size = 0.1,
                legendgroup = v,
                showlegend = False,
                line_color = c,
                line_width = 0.1,
                fillcolor = c,
                fill = "toself")
            fig.add_trace(locus)

In [6]:
#Legend drawing

def AddLegend(fig, value, color):
    for v,c in zip(value,color) :
        legend = go.Scatter(
            name = v,
            x = [0], 
            y = [0],
            marker_size = 0.01,
            line_color = c,
            line_width = 0.1,
            fillcolor = c,
            fill = "toself",
            hoverinfo = 'none')
        fig.add_trace(legend)

## Chromosome 1 exemple

In [8]:
#Get data
chrom_1_sense = getChromInfo('../SCERE.db', "1", 
                             """Primary_SGDID, Start_coordinate, Stop_coordinate, Feature_type, Feature_name, Description""")[0]

chrom_1_antisense = getChromInfo('../SCERE.db', "1", 
                                 """Primary_SGDID, Start_coordinate, Stop_coordinate, Feature_type, Feature_name, Description""")[1]

#display(chrom_1_sense.head())

#Figure's Layout
layout = go.Layout(plot_bgcolor='white')
fig = go.Figure(layout = layout)
fig.update_layout(xaxis_showgrid=False, yaxis_showgrid=False)
fig.update_xaxes(range=[-10000, 260000])
fig.update_yaxes(range=[-100, 100])

#Add chromosome
AddChromosome(fig, 0, 230218, 0, -20, 'lightgrey', '1')

# Add locus
chrom_1_sense.apply(AddLocus, axis = 1, args =(fig,
                                         0,
                                         -10,
                                         ["Feature_type","Feature_type","Feature_type"],
                                         ["CDS","centromere","telomere"],
                                         ['#B8AC77','black',"black"]))

chrom_1_antisense.apply(AddLocus, axis = 1, args =(fig,
                                            -11,
                                            -20,
                                            ["Feature_type","Feature_type","Feature_type"],
                                            ["CDS","centromere","telomere"],
                                            ['#B8AC77','black',"black"]))

# Add legend
AddLegend(fig,
          ["CDS","centromere","telomere"],
          ['#B8AC77','black',"black"])

fig.show()